In [1]:
import torch
from torch import optim
import torch.nn as nn
from torch.nn import MSELoss
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import torchvision.transforms as transforms
from torchvision.io import read_image
import numpy as np
from numpy import random
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import splitext
from glob import glob
from PIL import Image
from tqdm import tqdm
import logging
from unet_model import UNet

In [2]:
def imshow(img):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)

In [3]:
class TheDataset(Dataset):
    def __init__(self, interlaced_dir, gtruth_dir, scale=1):
        self.interlaced_dir = interlaced_dir
        self.gtruth_dir = gtruth_dir
        self.scale = scale
        self.transform = transforms.Compose([transforms.RandomCrop(256), transforms.ToTensor()])
        
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'

        self.ids = [splitext(file)[0] for file in listdir(interlaced_dir)]
        logging.info(f'Creating dataset with {len(self.ids)} examples')

    def __len__(self):
        return len(self.ids)

    @classmethod
    def preprocess(cls, pil_img, scale):
        pil_img = pil_img
        
        return pil_img

    def __getitem__(self, i):
        idx = self.ids[i]
        gtruth_file = glob(self.gtruth_dir + idx + '.*')
        interlaced_file = glob(self.interlaced_dir + idx + '.*')

        assert len(gtruth_file) == 1, \
            f'Either no mask or multiple masks found for the ID {idx}: {gtruth_file}'
        assert len(interlaced_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {interlaced_file}'
        gtruth = Image.open(gtruth_file[0])
        interlaced = Image.open(interlaced_file[0])

        assert interlaced.size == gtruth.size, \
            f'Image and mask {idx} should be the same size, but are {interlaced.size} and {gtruth.size}'
        
        seed = np.random.randint(2147483647) # make a seed with numpy generator 
        
        random.seed(seed) # apply this seed to img tranfsorms
        torch.manual_seed(seed) # needed for torchvision 0.7
        interlaced = self.transform(interlaced)
        interlaced = self.preprocess(interlaced, self.scale)
        
        random.seed(seed) # apply this seed to img tranfsorms
        torch.manual_seed(seed) # needed for torchvision 0.7
        gtruth = self.transform(gtruth)
        gtruth = self.preprocess(gtruth, self.scale)

        return interlaced, gtruth
    

In [4]:
interlaced_dir = "./dataset/interlaced/"
gtruth_dir = "./dataset/ground_truth/"
img_scale = 1

In [5]:
batch_size = 192
val_percent = 0.1

random.seed(23)
torch.manual_seed(23)

dataset = TheDataset(interlaced_dir, gtruth_dir, img_scale)
n_val = int(len(dataset) * val_percent)
n_train = len(dataset) - n_val
train, val = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
val_loader = DataLoader(val, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True, drop_last=True)
# for inter, truth in tqdm(train_loader):

#     print(np.array(inter[0].permute(1,2,0)))
#     imshow(np.array(inter[0].permute(1,2,0)))
#     imshow(np.array(truth[0].permute(1,2,0)))
#     break

In [6]:
def train_net(idx, net, lr):

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=1e-3)
    criterion = MSELoss()
    
    net.train()
    epoch_loss = 0

    ##################### TRAINING LOOP ########################
    
    for batch, (interlaced, truths) in enumerate(tqdm(train_loader)):
        interlaced = interlaced.to(device=device, dtype=torch.float32)
        truths = truths.to(device=device, dtype=torch.float32)

        net_pred = net(interlaced)
        loss = criterion(net_pred, truths)
        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 69 == 0:
            loss, current = loss.item(), batch * len(interlaced)
            print(f"loss: {loss}  [{current}/{n_train}]")

    print(f"Epoch {idx+1} loss: {epoch_loss/len(train_loader)}-------------------\n")

    ##################### VALIDATION LOOP ########################
    
    test_loss = 0
    with torch.no_grad():
        net.eval()
        for batch, (interlaced, truths) in enumerate(tqdm(val_loader)):

            interlaced = interlaced.to(device=device, dtype=torch.float32)
            truths = truths.to(device=device, dtype=torch.float32)

            net_pred = net(interlaced)
            test_loss += criterion(net_pred, truths).item()

    print(f"Test loss for epoch {idx+1}: {test_loss/len(val_loader)}-------------------\n")

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = UNet(n_channels=3, bilinear=True)
net = nn.DataParallel(net)
net.to(device=device)
lr_list = [1e-3, 1e-4, 1e-5, 1e-6]
for idx, lr in enumerate(lr_list):
    print(f"Epoch {idx+1}\n-------------------------------")
    train_net(idx=idx, net=net, lr=lr)
    torch.save(net, f'model-{idx+1}.pth')
    

  0%|          | 0/443 [00:00<?, ?it/s]

Epoch 1
-------------------------------


  0%|          | 1/443 [01:55<14:12:08, 115.67s/it]

loss: 0.5983479022979736  [0/84924]


 16%|█▌        | 70/443 [09:57<44:03,  7.09s/it]   

...

loss: 0.0006432349910028279  [79488/84924]


  0%|          | 0/49 [00:00<?, ?it/s]

Epoch 4 loss: 0.0006520167308093236-------------------



100%|██████████| 49/49 [06:40<00:00,  8.17s/it]  


Test loss for epoch 4: 0.0005256581388660992-------------------

